# Fulton Parcel Processing

In [1]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)

In [2]:
# Original names of variables, include these on data format sheet
original_vars = [
    'Taxyr', 'Parid', 'Situs Adrno', 'Situs Adrdir',
    'Situs Adrstr', 'Situs Adrsuf', 'Situs Adrsuf2',
    'Cityname', 'Class', 'Luc', 'Livunit',
    'Taxdist', 'Own1', 'Own2', 'Owner Adrno',
    'Owner Adradd', 'Owner Adrdir', 'Owner Adrstr',
    'Owner Adrsuf', 'Owner Adrsuf2', 'Cityname.1',
    'Statecode', 'Country', 'Unitno', 'Zip1', 'Reascd',
    'Aprland', 'Aprbldg', 'Revcode', 'Revreas', 'Aprtot',
    'D Yrblt', 'D Effyr', 'D Yrremod', 'D Grade', 'Sfla'
]
# New names of variables, 'final variable name' on data format sheet
new_var_map = {
    'Taxyr': 'tax_year', 'Parid': 'parcel_id', 'site_addr': 'site_addr',
    'Class': 'site_class_parcel', 'Luc': 'site_luc_parcel',
    'Taxdist': 'tax_district', 'Own1': 'owner_name_1', 
    'Own2': 'owner_name_2', 'Zip1': 'owner_zip', 'owner_addr': 'owner_addr',
    'Reascd': 'reas_cd', 'Revcode': 'rev_code',
    'Revreas': 'rev_reas', 'Aprtot': 'appr_total',
    'Aprland': 'appr_land', 'Aprbldg': 'appr_build', 'D Yrblt': 'year_built',
    'D Effyr': 'year_eff', 'D Yrremod': 'year_remodel', 'D Grade': 'grade',
    'Livunit': 'num_units', 'Sfla': 'building_area'
}
# New variable names ('final variable name' on data format sheet)
# with their associated datatype mapping
dtypes = {
    'tax_year': 'int16', 'parcel_id': 'string', 'site_addr': 'string',
    'site_class_parcel': 'string', 'site_luc_parcel': 'string',
    'tax_district': 'string', 'owner_name_1': 'string',
    'owner_name_2': 'string', 'owner_zip': 'string', 'owner_addr': 'string',
    'reas_cd': 'string', 'rev_code': 'string', 'rev_reas': 'string',
    'appr_total': 'float32', 'appr_land': 'float32', 'appr_build': 'float32',
    'year_built': 'int16', 'year_eff': 'int16',
    'year_remodel': 'int16', 'grade': 'string',
    'num_units': 'int16', 'building_area': 'float32'
}

## Read in Files

Get all file names in directory.

In [3]:
prefix_path = '../data/fulton/parcels/'
file_paths = os.listdir(prefix_path)
total_files = len(file_paths)
file_paths[:2] # printing out the first 2 files

['14_2010-2011.xlsx', '14_2012-2013.xlsx']

Loop through files and read them in. Store them in a list of dataframes. appendeded on a single file first.

In [4]:
dfs = []
count = 0

for file in file_paths:
    print(f'{count} / {total_files}')
    dfs.append(
        pd.read_excel(prefix_path + file)
    )
    count += 1

0 / 53
1 / 53
2 / 53
3 / 53
4 / 53
5 / 53
6 / 53
7 / 53
8 / 53
9 / 53
10 / 53
11 / 53
12 / 53
13 / 53
14 / 53
15 / 53
16 / 53
17 / 53
18 / 53
19 / 53
20 / 53
21 / 53
22 / 53
23 / 53
24 / 53
25 / 53
26 / 53
27 / 53
28 / 53
29 / 53
30 / 53
31 / 53
32 / 53
33 / 53
34 / 53
35 / 53
36 / 53


KeyboardInterrupt: 

## Append Files

Dropping extra vars.

Recommend dropping uneeded variables first if data size is large; the best way to do this is by only keeping variables we need because we don't need many.

We also want to remove the last row of each file (for Fulton, at least), because it simply contains a 'Count Distinct' value in the source data- this is not something we want.

In [ ]:
# Calculating total num of rows to verify next step
total_rows = 0

for index, df in enumerate(dfs):
    init = len(df.columns)
    dfs[index] = df[original_vars][:-1]
    total_rows += len(df)
    print(f'Dropped {init - len(df.columns)} columns')

Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns
Dropped 0 columns


Now all dfs should have the same columns. We can append.

In [ ]:
appended = pd.concat(dfs, ignore_index=True)

init_size = len(appended) # Take inital size of data for later calc

print(f'Appended has {len(appended)} rows; total rows calculated '
      + f'previously is {total_rows}')

Appended has 5986515 rows; total rows calculated previously is 5986568


Construct the derived variables (e.g. site_addr from the multiple site address variables in this data). However, before we can do this, we need to clean each of the columns we will join to create this variable.

More specifically, we need check how many None values there are in each column, then replace them with something that makes sense. This is because if you try to add a row with value None for one column (say address num) and "Main Street" for another, the result will be None.

In [ ]:
check_vars = ['Situs Adrno', 'Situs Adrdir','Situs Adrstr',
              'Situs Adrsuf', 'Situs Adrsuf2', 'Cityname',
              'Owner Adrno', 'Owner Adrstr', 'Owner Adradd',
              'Owner Adrdir', 'Owner Adrsuf', 'Owner Adrsuf2',
              'Cityname.1', 'Statecode', 'Unitno']

print("Percent NA by Variable ---")
for v in check_vars:
    pct_na = len(appended[appended[v].isna()]) / len(appended)* 100
    print(f'{v}: {pct_na:.2f}')

Percent NA by Variable ---
Situs Adrno: 0.01


Situs Adrdir: 99.94
Situs Adrstr: 0.00
Situs Adrsuf: 5.59
Situs Adrsuf2: 48.49
Cityname: 0.42
Owner Adrno: 4.02
Owner Adrstr: 0.42
Owner Adradd: 99.66
Owner Adrdir: 96.87
Owner Adrsuf: 8.39
Owner Adrsuf2: 61.92
Cityname.1: 0.40
Statecode: 0.41
Unitno: 80.62


Let's investigate some examples of NA for Cityname because, while insignificant, it is an important column.

In [ ]:
appended[appended['Cityname'].isna()].sample(2)

,Taxyr,Parid,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Situs Adrsuf2,Cityname,Class,Luc,Livunit,Taxdist,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,Owner Adrsuf2,Cityname.1,Statecode,Country,Unitno,Zip1,Reascd,Aprland,Aprbldg,Revcode,Revreas,Aprtot,D Yrblt,D Effyr,D Yrremod,D Grade,Sfla
1562315,2016.0,17 010900051023,140.0,NaN,ALDEN,AVE,NW,NaN,R3,106.0,1.0,5,AMERICAN FREEDOM INVESTMENTS LLC,NaN,2221.0,NaN,NaN,PEACHTREE,RD,NE,ATLANTA,GA,NaN,D106,30309,NaN,34600.0,124700.0,NaN,NaN,159300.0,2006.0,NaN,NaN,A-,827.0
5588981,2017.0,09F250201061526,5533.0,NaN,WAVERLY PARK,NaN,NaN,NaN,R3,100.0,1.0,55,D R HORTON INC,NaN,NaN,NaN,NaN,P O BOX 422357,NaN,NaN,ATLANTA,GA,NaN,NaN,30342,NaN,12000.0,0.0,3.0,NaN,12000.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
appended[appended['Situs Adrstr'].isna()].sample(1)

,Taxyr,Parid,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Situs Adrsuf2,Cityname,Class,Luc,Livunit,Taxdist,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,Owner Adrsuf2,Cityname.1,Statecode,Country,Unitno,Zip1,Reascd,Aprland,Aprbldg,Revcode,Revreas,Aprtot,D Yrblt,D Effyr,D Yrremod,D Grade,Sfla
2407906,2014.0,14 0029 LL0421,0.0,NaN,NaN,RD,SE,ATLANTA,C4,300,0.0,5,COOGLER SUSIE ALICE EXTRX,NaN,2780.0,NaN,NaN,JONESBORO,RD,SE,ATLANTA,GA,NaN,NaN,30354,NaN,149500.0,0.0,NaN,NaN,149500.0,NaN,NaN,NaN,NaN,NaN


Let's also investigate NA examples of 'Owner Adrno', 'Owner Adrstr', 'Cityname.1', and 'Statecode'

In [ ]:
appended[appended['Owner Adrno'].isna()].sample(2)

,Taxyr,Parid,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Situs Adrsuf2,Cityname,Class,Luc,Livunit,Taxdist,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,Owner Adrsuf2,Cityname.1,Statecode,Country,Unitno,Zip1,Reascd,Aprland,Aprbldg,Revcode,Revreas,Aprtot,D Yrblt,D Effyr,D Yrremod,D Grade,Sfla
1740118,2018.0,17 018600061309,0.0,NaN,BRICKTON,DR,NaN,ATLANTA,R3,100.0,0.0,5,FIRST CHOICE BUILDERS INC,NaN,NaN,NaN,NaN,P.O. BOX 250214,NaN,NaN,ATLANTA,GA,NaN,NaN,30325,FN,100.0,0.0,3.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN
5183149,2011.0,13 0193 LL1264,7529.0,NaN,BOWHEAD,CT,NaN,FUL,R3,100.0,0.0,55,LEGACY INVESTMENT GROUP LLC,NaN,NaN,NaN,NaN,P O BOX 191588,NaN,NaN,ATLANTA,GA,NaN,NaN,31119,E1,7300.0,0.0,3.0,OVR,7300.0,NaN,NaN,NaN,NaN,NaN


Looks like empty Adrno might be when the owner uses a P.O. Box; we definitely want to keep these.

In [ ]:
appended[appended['Owner Adrstr'].isna()].sample(2)

,Taxyr,Parid,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Situs Adrsuf2,Cityname,Class,Luc,Livunit,Taxdist,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,Owner Adrsuf2,Cityname.1,Statecode,Country,Unitno,Zip1,Reascd,Aprland,Aprbldg,Revcode,Revreas,Aprtot,D Yrblt,D Effyr,D Yrremod,D Grade,Sfla
3795861,2022.0,14 008700010691,0.0,NaN,MARY,ST,SW,ATLANTA,R3,100,0.0,5,PITTSBURGH COMMUNITY HOUSING LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68400.0,0.0,1.0,20,68400.0,NaN,NaN,NaN,NaN,NaN
5677411,2018.0,13 0133 LL4881,2250.0,NaN,BIGWOOD,TRL,NaN,SOUTH FULTON,R3,107.0,1.0,55,RAVEN PROPERTY GROUP LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNITED KINGDOM,NaN,NaN,FN,13500.0,58000.0,3.0,NaN,71500.0,2008.0,NaN,NaN,C,1660.0


Hard to determine anything. But we should just keep them for now.

In [ ]:
appended[appended['Cityname.1'].isna()].sample(2)

,Taxyr,Parid,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Situs Adrsuf2,Cityname,Class,Luc,Livunit,Taxdist,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,Owner Adrsuf2,Cityname.1,Statecode,Country,Unitno,Zip1,Reascd,Aprland,Aprbldg,Revcode,Revreas,Aprtot,D Yrblt,D Effyr,D Yrremod,D Grade,Sfla
3061279,2017.0,14F0012 LL0283U0268,405.0,NaN,FAIRBURN,RD,SW,NaN,C3,208,NaN,5,WILDWOOD PARK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,268,NaN,NaN,10900.0,17300.0,NaN,NaN,28200.0,2007.0,NaN,NaN,NaN,0.0
2857551,2016.0,14 007500040346,723.0,NaN,FORMWALT,ST,SW,ATL,R3,101,1.0,5,NEROKGSG LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BELGIUM,NaN,NaN,NaN,11600.0,43300.0,NaN,NaN,54900.0,2004.0,NaN,NaN,C,1806.0


Hard to determine anything. But we should just keep them for now.

In [ ]:
appended[appended['Statecode'].isna()].sample(2)

,Taxyr,Parid,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Situs Adrsuf2,Cityname,Class,Luc,Livunit,Taxdist,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,Owner Adrsuf2,Cityname.1,Statecode,Country,Unitno,Zip1,Reascd,Aprland,Aprbldg,Revcode,Revreas,Aprtot,D Yrblt,D Effyr,D Yrremod,D Grade,Sfla
2104905,2012.0,14 003500010447U0061,2980.0,NaN,JONESBORO,RD,SE,NaN,C3,208,NaN,5,OXFORD VILLAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,NaN,NaN,16300.0,12100.0,1.0,CP,28400.0,2007.0,NaN,NaN,NaN,0.0
66051,2011.0,14 008200030579,490.0,NaN,MARIETTA,ST,NW,ATLANTA,R3,110,1.0,05Z,ROBINSON CHEYNEY LYNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,NaN,NaN,39000.0,134900.0,1.0,20,173900.0,1945.0,1990.0,1997.0,B+,1069.0


Many observations without Statecode appear to be in other countries.

There are not any major problems here, so let's just fill all NA with empty strings ''. We need to convert all these variables to strings first though.

In [ ]:
print("Fill the above variables with empty strings when NA ---")

for v in check_vars:
    appended[v] = appended[v].astype('string').fillna('')

Fill the above variables with empty strings when NA ---


Construct the derived varibles

In [ ]:
appended['site_addr'] = (
    appended['Situs Adrno'] + ' ' +
    appended['Situs Adrdir'] + ' ' +
    appended['Situs Adrstr'] + ' ' +
    appended['Situs Adrsuf'] + ' ' +
    appended['Situs Adrsuf2'] + ' ' +
    appended['Cityname']
)

In [ ]:
appended['owner_addr'] = (
    appended['Owner Adrno'] + ' ' +
    appended['Owner Adrstr'] + ' ' +
    #appended['Cityname.1'] + ' ' +
    #appended['Statecode'] + ' ' +
    appended['Unitno']
)

Check the created variables look as expected

In [ ]:
appended[['site_addr', 'owner_addr']].sample(5)

,site_addr,owner_addr
3453590,751.0 CUSTER AVE SE ATLANTA,751.0 CUSTER
5715891,0.0 FAYETTEVILLE RD UNION CITY,7300.0 LESTER
40028,860.0 PEACHTREE ST ATLANTA,1101.0 UNIVERSITY #4
382973,352.0 GLENN ST SW ATLANTA,117.0 OAKWIND
669428,1555.0 OLYMPIAN WAY SW ATLANTA,


Clean the decimal points from the numbers

In [ ]:
appended['site_addr'] = appended['site_addr'].str.replace('.0','')

In [ ]:
appended['owner_addr'] = appended['owner_addr'].str.replace('.0','')

In [ ]:
appended[['site_addr', 'owner_addr']].sample(5)

,site_addr,owner_addr
2247887,3827 COLLIER DR NW ATLANTA,3827 COLLIER
4487404,745 FALLING ROCKS CT ROS,745 FALLING ROCKS
1703842,115 MARK TRL SANDY SPRINGS,115 MARK
3946870,0 BUICE RD JOHNS CREEK,5050 BUICE
540908,584 GLEN IRIS DR NE ATL,584 GLEN IRIS


Rename the variables and cast all the columns to the correct types

In [ ]:
appended.rename(columns=new_var_map, inplace=True)

In [ ]:
appended = appended[list(new_var_map.values())]

In [ ]:
appended.columns

Index(['tax_year', 'parcel_id', 'site_addr', 'site_class_parcel',
       'site_luc_parcel', 'tax_district', 'owner_name_1', 'owner_name_2',
       'owner_zip', 'owner_addr', 'reas_cd', 'rev_code', 'rev_reas',
       'appr_total', 'appr_land', 'appr_build', 'year_built', 'year_eff',
       'year_remodel', 'grade', 'num_units', 'building_area'],
      dtype='object')

When you try to cast, expect there to be problems. It is likely that some of the vars will have values that cannot be easily casted (e.g. 'A' cannot be casted to an Int). You will need to resolve these for each column.

In [ ]:
#appended = appended.astype(dtypes)

Clean nan values in 'owner_zip'

In [ ]:
len(appended[appended['owner_zip'].isna()]) / len(appended) * 100

0.4649616680155315

In [ ]:
appended[appended['owner_zip'].isna()].sample(3)

,tax_year,parcel_id,site_addr,site_class_parcel,site_luc_parcel,tax_district,owner_name_1,owner_name_2,owner_zip,owner_addr,reas_cd,rev_code,rev_reas,appr_total,appr_land,appr_build,year_built,year_eff,year_remodel,grade,num_units,building_area
2907694,2016.0,14F0012 LL0234U0088,400 FAIRBURN RD SW,C3,208,5,FAIRBURN TOWNHOUSES,NaN,NaN,88,NaN,NaN,NaN,28700.0,16700.0,12000.0,2007.0,NaN,NaN,NaN,NaN,0.0
3035766,2017.0,14 014400010023,0 ELINOR PL NW ATL,R3,100,5,BROWN LINA W MRS,NaN,NaN,,NaN,NaN,NaN,16200.0,16200.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN
2550625,2013.0,14F0012 LL0283U0239,405 FAIRBURN RD SW,C3,208,5,WILDWOOD PARK,NaN,NaN,239,NaN,1.0,CP,30600.0,12800.0,17800.0,2007.0,NaN,NaN,NaN,NaN,0.0


It is an insignficant portion of the data, and quite important, so lets drop rows with None values for owner_zip

In [ ]:
#appended = appended[~appended['owner_zip'].isna()]
appended['owner_zip'].fillna('0', inplace=True)

Let's investigate year_built

In [ ]:
len(appended[appended['year_built'].isna()]) / len(appended) * 100

12.287349150549193

Significant number of None values. Let's replace them with 0, and same for the other year columns.

In [ ]:
appended['year_built'].fillna('0', inplace=True)
appended['year_eff'].fillna('0', inplace=True)
appended['year_remodel'].fillna('0', inplace=True)

Investigate 'num_units'

In [ ]:
len(appended[appended['num_units'].isna()]) / len(appended) * 100

0.9317440948531825

In [ ]:
appended[appended['num_units'].isna()].sample(5)

,tax_year,parcel_id,site_addr,site_class_parcel,site_luc_parcel,tax_district,owner_name_1,owner_name_2,owner_zip,owner_addr,reas_cd,rev_code,rev_reas,appr_total,appr_land,appr_build,year_built,year_eff,year_remodel,grade,num_units,building_area
2130919,2011.0,14 005400040242,0 CAPITOL AVE SE ATLANTA,C3,339,05P,DELTA PARKING INC,NaN,30324,1024 LINDBERG,NaN,3.0,80,74600.0,65600.0,9000.0,0,0,0,NaN,NaN,NaN
1906303,2020.0,17 0221 LL2863,1377 LIBERTY PKWY ATLANTA,R3,107.0,5,PINCU DOUGLAS,NaN,30318,1377 LIBERTY,RF,NaN,NaN,280900.0,42300.0,238600.0,2006.0,0,0,B,NaN,1326.0
3767533,2022.0,14 0029 LL0454U0298,2980 JONESBORO RD SE ATLANTA,C3,208,5,OXFORD VILLAGE,NaN,0,298,NaN,NaN,NaN,17800.0,0.0,17800.0,2007.0,0,0,NaN,NaN,0.0
912251,2021.0,14 009000011231,0 GRANT ST SW ATLANTA,R3,100.0,05R,SMITH ROBERT FLOYD &,THOMAS NICOLE,30252,126 PINEHAVEN,NaN,NaN,NaN,97200.0,97200.0,0.0,0,0,0,NaN,NaN,NaN
2550842,2014.0,14F0012 LL0291U0197,400 FAIRBURN RD SW,C3,208,5,FAIRBURN TOWNE HOUSES,NaN,0,197,NaN,1.0,CP,32300.0,1400.0,30900.0,2007.0,0,0,NaN,NaN,0.0


Let's fill these with 0.

In [ ]:
appended['num_units'].fillna('0', inplace=True)

In [ ]:
appended = appended[appended['tax_year'].str.count("([a-zA-Z])") <= 1]

In [ ]:
appended = appended.astype(dtypes)

Check the datatypes

In [ ]:
appended.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tax_year           0 non-null      int16  
 1   parcel_id          0 non-null      string 
 2   site_addr          0 non-null      string 
 3   site_class_parcel  0 non-null      string 
 4   site_luc_parcel    0 non-null      string 
 5   tax_district       0 non-null      string 
 6   owner_name_1       0 non-null      string 
 7   owner_name_2       0 non-null      string 
 8   owner_zip          0 non-null      string 
 9   owner_addr         0 non-null      string 
 10  reas_cd            0 non-null      string 
 11  rev_code           0 non-null      string 
 12  rev_reas           0 non-null      string 
 13  appr_total         0 non-null      float32
 14  appr_land          0 non-null      float32
 15  appr_build         0 non-null      float32
 16  year_built         0 non-null      int16  


Export the data

In [ ]:
appended.to_csv('../output/all_parcels_fulton_2011_2022.csv')
appended.to_parquet('../output/all_parcels_fulton_2011_2022.parquet')